In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
# !pip install -q -U keras-tuner

import kerastuner as kt

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [5]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
    model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer 
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

    model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                  loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                  metrics = ['accuracy'])

    return model

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')    

In [7]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

In [8]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



In [9]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10


1567/1875 [========================>.....] - ETA: 0s - loss: 2.4355 - accuracy: 0.28 - ETA: 11s - loss: 1.8469 - accuracy: 0.429 - ETA: 9s - loss: 1.4085 - accuracy: 0.562 - ETA: 10s - loss: 1.2691 - accuracy: 0.584 - ETA: 9s - loss: 1.1369 - accuracy: 0.617 - ETA: 11s - loss: 1.1228 - accuracy: 0.622 - ETA: 11s - loss: 1.0818 - accuracy: 0.636 - ETA: 11s - loss: 1.0268 - accuracy: 0.652 - ETA: 10s - loss: 0.9502 - accuracy: 0.678 - ETA: 9s - loss: 0.8855 - accuracy: 0.697 - ETA: 9s - loss: 0.8622 - accuracy: 0.70 - ETA: 10s - loss: 0.8585 - accuracy: 0.706 - ETA: 11s - loss: 0.8573 - accuracy: 0.707 - ETA: 12s - loss: 0.8541 - accuracy: 0.708 - ETA: 11s - loss: 0.8332 - accuracy: 0.714 - ETA: 12s - loss: 0.8300 - accuracy: 0.715 - ETA: 12s - loss: 0.8261 - accuracy: 0.717 - ETA: 13s - loss: 0.8252 - accuracy: 0.718 - ETA: 13s - loss: 0.8131 - accuracy: 0.721 - ETA: 14s - loss: 0.8103 - accuracy: 0.722 - ETA: 14s - loss: 0.8015 - accuracy: 0.725 - ETA: 13s - loss: 0.7852 - accuracy: 0.

1875/1875 [==============================] - ETA: 2s - loss: 0.4999 - accuracy: 0.82 - ETA: 2s - loss: 0.4996 - accuracy: 0.82 - ETA: 2s - loss: 0.4992 - accuracy: 0.82 - ETA: 2s - loss: 0.4992 - accuracy: 0.82 - ETA: 2s - loss: 0.4978 - accuracy: 0.82 - ETA: 1s - loss: 0.4966 - accuracy: 0.82 - ETA: 1s - loss: 0.4962 - accuracy: 0.82 - ETA: 1s - loss: 0.4948 - accuracy: 0.82 - ETA: 1s - loss: 0.4939 - accuracy: 0.82 - ETA: 1s - loss: 0.4935 - accuracy: 0.82 - ETA: 1s - loss: 0.4919 - accuracy: 0.82 - ETA: 1s - loss: 0.4908 - accuracy: 0.82 - ETA: 1s - loss: 0.4905 - accuracy: 0.82 - ETA: 1s - loss: 0.4901 - accuracy: 0.82 - ETA: 1s - loss: 0.4894 - accuracy: 0.82 - ETA: 1s - loss: 0.4892 - accuracy: 0.82 - ETA: 0s - loss: 0.4890 - accuracy: 0.82 - ETA: 0s - loss: 0.4888 - accuracy: 0.82 - ETA: 0s - loss: 0.4888 - accuracy: 0.82 - ETA: 0s - loss: 0.4886 - accuracy: 0.82 - ETA: 0s - loss: 0.4883 - accuracy: 0.82 - ETA: 0s - loss: 0.4869 - accuracy: 0.82 - ETA: 0s - loss: 0.4865 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.2317 - accuracy: 0.90 - ETA: 9s - loss: 0.3900 - accuracy: 0.85 - ETA: 7s - loss: 0.3947 - accuracy: 0.86 - ETA: 7s - loss: 0.3878 - accuracy: 0.85 - ETA: 9s - loss: 0.3805 - accuracy: 0.86 - ETA: 13s - loss: 0.3813 - accuracy: 0.862 - ETA: 15s - loss: 0.3792 - accuracy: 0.863 - ETA: 16s - loss: 0.3809 - accuracy: 0.861 - ETA: 18s - loss: 0.3752 - accuracy: 0.862 - ETA: 19s - loss: 0.3760 - accuracy: 0.863 - ETA: 21s - loss: 0.3794 - accuracy: 0.863 - ETA: 22s - loss: 0.3747 - accuracy: 0.864 - ETA: 23s - loss: 0.3726 - accuracy: 0.864 - ETA: 24s - loss: 0.3715 - accuracy: 0.865 - ETA: 25s - loss: 0.3707 - accuracy: 0.864 - ETA: 25s - loss: 0.3660 - accuracy: 0.867 - ETA: 24s - loss: 0.3703 - accuracy: 0.866 - ETA: 24s - loss: 0.3713 - accuracy: 0.865 - ETA: 24s - loss: 0.3741 - accuracy: 0.865 - ETA: 25s - loss: 0.3722 - accuracy: 0.866 - ETA: 27s - loss: 0.3720 - accuracy: 0.865 - ETA: 29s - loss: 0.3767 - accuracy: 0.865

1875/1875 [==============================] - ETA: 0s - loss: 0.4898 - accuracy: 0.84 - ETA: 5s - loss: 0.3801 - accuracy: 0.85 - ETA: 4s - loss: 0.3426 - accuracy: 0.87 - ETA: 6s - loss: 0.3509 - accuracy: 0.86 - ETA: 6s - loss: 0.3606 - accuracy: 0.86 - ETA: 5s - loss: 0.3510 - accuracy: 0.86 - ETA: 5s - loss: 0.3522 - accuracy: 0.86 - ETA: 4s - loss: 0.3434 - accuracy: 0.87 - ETA: 4s - loss: 0.3335 - accuracy: 0.87 - ETA: 5s - loss: 0.3377 - accuracy: 0.87 - ETA: 5s - loss: 0.3360 - accuracy: 0.87 - ETA: 5s - loss: 0.3270 - accuracy: 0.87 - ETA: 4s - loss: 0.3298 - accuracy: 0.88 - ETA: 4s - loss: 0.3404 - accuracy: 0.87 - ETA: 4s - loss: 0.3432 - accuracy: 0.87 - ETA: 4s - loss: 0.3416 - accuracy: 0.87 - ETA: 3s - loss: 0.3398 - accuracy: 0.87 - ETA: 3s - loss: 0.3379 - accuracy: 0.87 - ETA: 4s - loss: 0.3383 - accuracy: 0.87 - ETA: 4s - loss: 0.3360 - accuracy: 0.87 - ETA: 4s - loss: 0.3357 - accuracy: 0.87 - ETA: 4s - loss: 0.3358 - accuracy: 0.87 - ETA: 4s - loss: 0.3349 - accura

...

1875/1875 [==============================] - ETA: 5s - loss: 0.3073 - accuracy: 0.88 - ETA: 5s - loss: 0.3074 - accuracy: 0.88 - ETA: 5s - loss: 0.3075 - accuracy: 0.88 - ETA: 4s - loss: 0.3071 - accuracy: 0.88 - ETA: 4s - loss: 0.3074 - accuracy: 0.88 - ETA: 4s - loss: 0.3066 - accuracy: 0.88 - ETA: 3s - loss: 0.3061 - accuracy: 0.88 - ETA: 3s - loss: 0.3064 - accuracy: 0.88 - ETA: 3s - loss: 0.3054 - accuracy: 0.88 - ETA: 2s - loss: 0.3050 - accuracy: 0.88 - ETA: 2s - loss: 0.3050 - accuracy: 0.88 - ETA: 2s - loss: 0.3047 - accuracy: 0.88 - ETA: 2s - loss: 0.3046 - accuracy: 0.88 - ETA: 2s - loss: 0.3039 - accuracy: 0.88 - ETA: 2s - loss: 0.3034 - accuracy: 0.88 - ETA: 1s - loss: 0.3032 - accuracy: 0.88 - ETA: 1s - loss: 0.3034 - accuracy: 0.88 - ETA: 1s - loss: 0.3032 - accuracy: 0.88 - ETA: 1s - loss: 0.3031 - accuracy: 0.88 - ETA: 1s - loss: 0.3031 - accuracy: 0.88 - ETA: 1s - loss: 0.3029 - accuracy: 0.88 - ETA: 1s - loss: 0.3029 - accuracy: 0.88 - ETA: 1s - loss: 0.3033 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.1516 - accuracy: 0.96 - ETA: 2s - loss: 0.2747 - accuracy: 0.90 - ETA: 3s - loss: 0.2807 - accuracy: 0.89 - ETA: 3s - loss: 0.2729 - accuracy: 0.89 - ETA: 2s - loss: 0.2676 - accuracy: 0.90 - ETA: 3s - loss: 0.2642 - accuracy: 0.90 - ETA: 3s - loss: 0.2646 - accuracy: 0.90 - ETA: 4s - loss: 0.2659 - accuracy: 0.89 - ETA: 4s - loss: 0.2646 - accuracy: 0.90 - ETA: 4s - loss: 0.2672 - accuracy: 0.89 - ETA: 4s - loss: 0.2685 - accuracy: 0.89 - ETA: 4s - loss: 0.2673 - accuracy: 0.89 - ETA: 4s - loss: 0.2699 - accuracy: 0.89 - ETA: 4s - loss: 0.2714 - accuracy: 0.89 - ETA: 4s - loss: 0.2735 - accuracy: 0.89 - ETA: 4s - loss: 0.2762 - accuracy: 0.89 - ETA: 4s - loss: 0.2763 - accuracy: 0.89 - ETA: 4s - loss: 0.2768 - accuracy: 0.89 - ETA: 3s - loss: 0.2777 - accuracy: 0.89 - ETA: 3s - loss: 0.2752 - accuracy: 0.89 - ETA: 3s - loss: 0.2758 - accuracy: 0.89 - ETA: 3s - loss: 0.2756 - accuracy: 0.89 - ETA: 3s - loss: 0.2760 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.1752 - accuracy: 0.87 - ETA: 12s - loss: 0.2695 - accuracy: 0.909 - ETA: 7s - loss: 0.2452 - accuracy: 0.914 - ETA: 9s - loss: 0.2406 - accuracy: 0.91 - ETA: 11s - loss: 0.2452 - accuracy: 0.909 - ETA: 8s - loss: 0.2442 - accuracy: 0.909 - ETA: 7s - loss: 0.2427 - accuracy: 0.91 - ETA: 7s - loss: 0.2467 - accuracy: 0.90 - ETA: 6s - loss: 0.2447 - accuracy: 0.90 - ETA: 5s - loss: 0.2466 - accuracy: 0.90 - ETA: 5s - loss: 0.2449 - accuracy: 0.90 - ETA: 5s - loss: 0.2441 - accuracy: 0.90 - ETA: 6s - loss: 0.2424 - accuracy: 0.90 - ETA: 5s - loss: 0.2373 - accuracy: 0.91 - ETA: 6s - loss: 0.2369 - accuracy: 0.91 - ETA: 6s - loss: 0.2344 - accuracy: 0.91 - ETA: 6s - loss: 0.2320 - accuracy: 0.91 - ETA: 6s - loss: 0.2379 - accuracy: 0.91 - ETA: 6s - loss: 0.2381 - accuracy: 0.91 - ETA: 6s - loss: 0.2394 - accuracy: 0.91 - ETA: 6s - loss: 0.2404 - accuracy: 0.91 - ETA: 6s - loss: 0.2412 - accuracy: 0.90 - ETA: 6s - loss: 0.2422 - 

1875/1875 [==============================] - ETA: 0s - loss: 0.3493 - accuracy: 0.90 - ETA: 6s - loss: 0.2566 - accuracy: 0.90 - ETA: 3s - loss: 0.2173 - accuracy: 0.91 - ETA: 3s - loss: 0.2253 - accuracy: 0.91 - ETA: 3s - loss: 0.2260 - accuracy: 0.91 - ETA: 2s - loss: 0.2171 - accuracy: 0.91 - ETA: 2s - loss: 0.2165 - accuracy: 0.91 - ETA: 2s - loss: 0.2246 - accuracy: 0.91 - ETA: 3s - loss: 0.2236 - accuracy: 0.91 - ETA: 2s - loss: 0.2208 - accuracy: 0.91 - ETA: 2s - loss: 0.2220 - accuracy: 0.91 - ETA: 2s - loss: 0.2268 - accuracy: 0.91 - ETA: 2s - loss: 0.2284 - accuracy: 0.91 - ETA: 2s - loss: 0.2302 - accuracy: 0.91 - ETA: 2s - loss: 0.2311 - accuracy: 0.91 - ETA: 2s - loss: 0.2325 - accuracy: 0.91 - ETA: 2s - loss: 0.2323 - accuracy: 0.91 - ETA: 2s - loss: 0.2306 - accuracy: 0.91 - ETA: 2s - loss: 0.2305 - accuracy: 0.91 - ETA: 2s - loss: 0.2309 - accuracy: 0.91 - ETA: 3s - loss: 0.2305 - accuracy: 0.91 - ETA: 2s - loss: 0.2326 - accuracy: 0.91 - ETA: 2s - loss: 0.2317 - accura

1875/1875 [==============================] - ETA: 0s - loss: 0.2755 - accuracy: 0.87 - ETA: 11s - loss: 0.1943 - accuracy: 0.910 - ETA: 6s - loss: 0.2382 - accuracy: 0.914 - ETA: 5s - loss: 0.2238 - accuracy: 0.92 - ETA: 5s - loss: 0.2171 - accuracy: 0.91 - ETA: 6s - loss: 0.2291 - accuracy: 0.91 - ETA: 5s - loss: 0.2235 - accuracy: 0.91 - ETA: 5s - loss: 0.2245 - accuracy: 0.91 - ETA: 5s - loss: 0.2212 - accuracy: 0.91 - ETA: 4s - loss: 0.2206 - accuracy: 0.91 - ETA: 5s - loss: 0.2209 - accuracy: 0.91 - ETA: 5s - loss: 0.2223 - accuracy: 0.91 - ETA: 5s - loss: 0.2220 - accuracy: 0.91 - ETA: 5s - loss: 0.2244 - accuracy: 0.91 - ETA: 6s - loss: 0.2243 - accuracy: 0.91 - ETA: 6s - loss: 0.2247 - accuracy: 0.91 - ETA: 6s - loss: 0.2248 - accuracy: 0.91 - ETA: 6s - loss: 0.2242 - accuracy: 0.91 - ETA: 6s - loss: 0.2241 - accuracy: 0.91 - ETA: 6s - loss: 0.2206 - accuracy: 0.91 - ETA: 6s - loss: 0.2206 - accuracy: 0.91 - ETA: 6s - loss: 0.2218 - accuracy: 0.91 - ETA: 6s - loss: 0.2216 - acc